In [34]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import optimizers, losses, activations, models
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.python.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, Flatten
from sklearn.metrics import f1_score, accuracy_score


In [35]:
df_train = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [36]:
def identity_block(x,filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=5, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)  
    x = tf.keras.layers.Activation('relu')(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [37]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[0], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    x = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv1D(filter[1], kernel_size=3, padding = 'same')(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [38]:
def get_model1():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[32,32])
    img_shortcut = identity_block(img_shortcut,[32,32])

    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = Flatten()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [39]:
model1 = get_model1()
file_path = "Resnet_cnn_mitbih1.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_264 (Conv1D)             (None, 187, 16)      96          input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_265 (Conv1D)             (None, 187, 16)      1296        conv1d_264[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_6 (MaxPooling1D)  (None, 93, 16)       0           conv1d_265[0][0]                 
____________________________________________________________________________________________

In [40]:
model1.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.load_weights(file_path)

pred_test = model1.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.97202, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 165s - loss: 0.1681 - acc: 0.9546 - val_loss: 0.0997 - val_acc: 0.9720
Epoch 2/1000

Epoch 00002: val_acc improved from 0.97202 to 0.97533, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 142s - loss: 0.0963 - acc: 0.9724 - val_loss: 0.0864 - val_acc: 0.9753
Epoch 3/1000

Epoch 00003: val_acc improved from 0.97533 to 0.97750, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 140s - loss: 0.0792 - acc: 0.9764 - val_loss: 0.0787 - val_acc: 0.9775
Epoch 4/1000

Epoch 00004: val_acc improved from 0.97750 to 0.97796, saving model to Resnet_cnn_mitbih1.h5
78798/78798 - 142s - loss: 0.0676 - acc: 0.9800 - val_loss: 0.0724 - val_acc: 0.9780
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.97796
78798/78798 - 147s - loss: 0.0592 - acc: 0.9825 - val_loss: 0.0729 - val_acc: 0.9768
Epoch 6/1000

Epoch 00006: val_acc di

In [41]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9198314392793776 
Test accuracy score : 0.985108715512516 


In [42]:
def get_model2():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = convolutional_block(img_1,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [43]:
model2 = get_model2()
file_path = "Resnet_cnn_mitbih2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_277 (Conv1D)             (None, 187, 16)      96          input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_278 (Conv1D)             (None, 187, 16)      1296        conv1d_277[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 93, 16)       0           conv1d_278[0][0]                 
____________________________________________________________________________________________

In [44]:
model2.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.load_weights(file_path)

pred_test2 = model2.predict(X_test)
pred_test2 = np.argmax(pred_test2, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.96836, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 111s - loss: 0.2308 - acc: 0.9373 - val_loss: 0.1254 - val_acc: 0.9684
Epoch 2/1000

Epoch 00002: val_acc improved from 0.96836 to 0.97145, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 106s - loss: 0.1395 - acc: 0.9617 - val_loss: 0.1063 - val_acc: 0.9714
Epoch 3/1000

Epoch 00003: val_acc improved from 0.97145 to 0.97213, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 106s - loss: 0.1217 - acc: 0.9665 - val_loss: 0.1079 - val_acc: 0.9721
Epoch 4/1000

Epoch 00004: val_acc improved from 0.97213 to 0.97339, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 109s - loss: 0.1095 - acc: 0.9702 - val_loss: 0.0982 - val_acc: 0.9734
Epoch 5/1000

Epoch 00005: val_acc improved from 0.97339 to 0.97487, saving model to Resnet_cnn_mitbih2.h5
78798/78798 - 108s - loss: 0.1036 - acc: 0.9722 - val_loss: 0.0902 - val_acc: 0

In [45]:
f1 = f1_score(Y_test, pred_test2, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test2)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9065457524254107 
Test accuracy score : 0.9841951397770875 


In [48]:
def get_model3():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_1 = tf.keras.layers.Convolution1D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.Convolution1D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
    
    img_shortcut = identity_block(img_1,[32,32])
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_shortcut)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [49]:
model3 = get_model3()
file_path = "Resnet_cnn_mitbih3.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_290 (Conv1D)             (None, 187, 16)      96          input_10[0][0]                   
__________________________________________________________________________________________________
conv1d_291 (Conv1D)             (None, 187, 16)      1296        conv1d_290[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_10 (MaxPooling1D) (None, 93, 16)       0           conv1d_291[0][0]                 
____________________________________________________________________________________________

In [50]:
model3.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.load_weights(file_path)

pred_test3 = model3.predict(X_test)
pred_test3 = np.argmax(pred_test3, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from -inf to 0.95215, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 80s - loss: 0.2529 - acc: 0.9302 - val_loss: 0.1679 - val_acc: 0.9521
Epoch 2/1000

Epoch 00002: val_acc improved from 0.95215 to 0.96722, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 74s - loss: 0.1419 - acc: 0.9613 - val_loss: 0.1178 - val_acc: 0.9672
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.96722
78798/78798 - 74s - loss: 0.1151 - acc: 0.9693 - val_loss: 0.1191 - val_acc: 0.9633
Epoch 4/1000

Epoch 00004: val_acc improved from 0.96722 to 0.97682, saving model to Resnet_cnn_mitbih3.h5
78798/78798 - 75s - loss: 0.1018 - acc: 0.9721 - val_loss: 0.0862 - val_acc: 0.9768
Epoch 5/1000

Epoch 00005: val_acc did not improve from 0.97682
78798/78798 - 74s - loss: 0.0912 - acc: 0.9747 - val_loss: 0.0841 - val_acc: 0.9765
Epoch 6/1000

Epoch 00006: val_acc improved from 0.97682 to 0.97921, saving model to 

In [51]:
f1 = f1_score(Y_test, pred_test3, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test3)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9171564726806969 
Test accuracy score : 0.9854741458066874 
